In [92]:
import pandas as pd
import csv
import numpy as np
np.random.seed(220718)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
from sklearn.tree import plot_tree

In [93]:
# Lectura del csv para la tabla con todas las métricas
nodes = pd.read_csv("../tablas/tableWithAllAtributes.csv")

In [94]:
nodes.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [95]:
# PREPARACIÓN DE LOS DATOS PARA LA TABLA CON TODOS LOS ATRIBUTOS
atributos_discretos = ['name']
atributos_continuos = ['degree_centrality','closeness_centrality','betweenness_centrality','clustering_coefficient','Square clustering','triangles','greedy_modularity_communities','Core number','asyn_lpa_communities']
atributos = nodes.loc[:,['id_node'] + atributos_discretos + atributos_continuos]
atributos.head()

,id_node,name,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,Square clustering,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.000212,0.294956,1.149733e-06,0.178571,0.072344,0.000062,0.002227,0.151515,0.0
2,2,JpMCarrilho,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,SuhwanCha,0.000133,0.278718,5.316292e-05,0.000000,0.019178,0.000000,0.004454,0.090909,0.0
4,4,sunilangadi2,0.000053,0.243084,6.134318e-09,0.000000,0.000000,0.000000,0.011136,0.030303,0.0


In [96]:
# Objetivo
objetivo = nodes['ml_target']
objetivo.head()

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: ml_target, dtype: float64

In [97]:
# Codificación
codificador_atributos_discretos = OrdinalEncoder() # Crear una instancia de la clase correspondiente
codificador_atributos_discretos.fit(atributos[atributos_discretos]) # Usar el método fit para ajustar a los datos los parámetros de la codificación

# Vemos información sobre los atributos discretos
print("///Información sobre los atributos discretos///")
print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
print()
print('Nombres de los atributos detectados:')
print(f'{codificador_atributos_discretos.feature_names_in_}')
print()
print('Categorías detectadas de cada atributo:')
for atributo, categorías in zip(
    codificador_atributos_discretos.feature_names_in_,
    codificador_atributos_discretos.categories_):
    print(f'{atributo}: {categorías}')

# Ahora aplicamos el método transform para codificar los datos
atributos[atributos_discretos] = codificador_atributos_discretos.transform(
    atributos[atributos_discretos]
)
print()
print("///Tabla con la columna name cambiada a números///")
print(atributos.head())


///Información sobre los atributos discretos///
Número de atributos detectados: 1

Nombres de los atributos detectados:
['name']

Categorías detectadas de cada atributo:
name: ['007arunwilson' '007jedgar' '00Kai0' ... 'timothykimemia' 'timoxley'
 'timqian']

///Tabla con la columna name cambiada a números///
   id_node     name  degree_centrality  closeness_centrality  \
0        0   2325.0           0.000027              0.275005   
1        1  35055.0           0.000212              0.294956   
2        2   4022.0           0.000027              0.261845   
3        3   7220.0           0.000133              0.278718   
4        4  36547.0           0.000053              0.243084   

   betweenness_centrality  clustering_coefficient  Square clustering  \
0            0.000000e+00                0.000000           0.000000   
1            1.149733e-06                0.178571           0.072344   
2            0.000000e+00                0.000000           0.000000   
3            5.31

In [98]:
# Dividimos los conjuntos de prueba y entrenamiento
'''''
#División 1
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.1,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)
'''''
#División 2
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)
'''''
#División 3
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.3,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)
'''''

"''\n#División 3\n(atributos_entrenamiento, atributos_prueba,\n objetivo_entrenamiento, objetivo_prueba) = train_test_split(\n        # Conjuntos de datos a dividir, usando los mismos índices para ambos\n        atributos, objetivo,\n        # Tamaño del conjunto de prueba (20 % en este caso)\n        test_size=.3,\n        # Estratificación según la distribución de clases en el atributo objetivo\n        stratify=objetivo)\n"

In [99]:
clasificador_CART = DecisionTreeClassifier()
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'max_depth': range(3, 16),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'min_samples_split': range(5, 26, 5)
}

In [100]:
búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(3, 16),
                         'min_samples_split': range(5, 26, 5)},
             scoring='recall')

In [101]:
búsqueda_en_rejilla.best_params_

{'max_depth': 3, 'min_samples_split': 5}

In [102]:
búsqueda_en_rejilla.best_score_

0.5736119614232579

In [103]:
clasificador_CART = DecisionTreeClassifier(
    max_depth=3,  # Máxima profundidad del árbol
    min_samples_split=5  # Mínimo número de ejemplos para poder particionar
)

In [104]:
resultados_validación_cruzada = cross_validate(clasificador_CART,
                                               atributos_entrenamiento,
                                               objetivo_entrenamiento,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzada

{'fit_time': array([0.09327817, 0.08078384, 0.07081318, 0.0629518 , 0.06674504,
        0.06242037, 0.06241345, 0.06318665, 0.0678184 , 0.06285715]),
 'score_time': array([0.00698137, 0.00598407, 0.0039897 , 0.00598502, 0.0039897 ,
        0.00371075, 0.00299025, 0.00598383, 0.0039897 , 0.00398707]),
 'test_score': array([0.56867779, 0.59563543, 0.57894737, 0.55840822, 0.58279846,
        0.58151476, 0.54172015, 0.59178434, 0.57381258, 0.56282051])}

In [105]:
resultados_validación_cruzada['test_score'].mean()

0.5736119614232579